In [167]:
import pandas as pd
import numpy as np

In [168]:
#Paths to files
encounter_path = "C:\\Users\\winsk\\Desktop\\UTSW Data\\df_encounter.csv"
labs_path = "C:\\Users\\winsk\\Desktop\\UTSW Data\\df_labs.csv"
vitals_path = "C:\\Users\\winsk\\Desktop\\UTSW Data\\df_vitals.csv"
outcome_path = "C:\\Users\\winsk\\Desktop\\UTSW Data\\df_outcome.csv"
cancer_path = "C:\\Users\\winsk\\Desktop\\UTSW Data\\df_cancer.csv"

In [203]:
#Read in dataframes
encounter_df = pd.read_csv(encounter_path)
labs_df = pd.read_csv(labs_path) #bnp and triponin
vitals_df = pd.read_csv(vitals_path)
outcome_df = pd.read_csv(outcome_path)
cancer_df = pd.read_csv(cancer_path)

In [204]:
def merge_cancer(enc_df, cnc_df):
    #Make modifications to cancer df
    mod_cancer_df = cnc_df.copy()
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Missing Remission Date', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Previously Positive', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Not Documented', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Not documented', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'No Cancer', 'cancer_at_enc'] = '1'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown', 'cancer_at_enc'] = '2'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Cancer', 'cancer_at_enc'] = '3'
    mod_cancer_df['cancer_at_enc'] = mod_cancer_df['cancer_at_enc'].astype(int)
    mod_cancer_df.drop_duplicates(['PATIENT_ID', 'HSP_ENC'])
    
    #Take only cancer status column with max value
    drop_cancer_df = mod_cancer_df[['HSP_ENC', 'cancer_at_enc']]
    drop_cancer_df = drop_cancer_df.groupby('HSP_ENC',group_keys=False).apply(lambda x: x.loc[x['cancer_at_enc']==x['cancer_at_enc'].max()])

    #Merge with encounter df and drop dups
    mergeRes = pd.merge(enc_df, drop_cancer_df, on='HSP_ENC', how='left')
    mergeRes = mergeRes.drop_duplicates('HSP_ENC')

    #Replace NaN in encounter df with 0 (Never had cancer)
    mergeRes['cancer_at_enc'].fillna(0, inplace=True)
    
    #Renaming the cancer column
    mergeRes.rename(columns={'cancer_at_enc': 'CANCER_RANK'}, inplace=True)
    
    return mergeRes

In [207]:
out = merge_cancer(encounter_df, cancer_df)

In [209]:
cancer_grp = out.groupby(['CANCER_RANK']).agg('count')
cancer_grp_count = cancer_grp.sort_values(['HSP_ENC'], ascending=True)
cancer_grp_count.iloc[:,1]

CANCER_RANK
1.0      75
2.0     172
3.0     366
0.0    1029
Name: HSP_ENC, dtype: int64